In [ ]:
# Install library for hyperparameter optimisation
!pip install keras-tuner -q

# Import
import pandas as pd
import numpy as np
import pickle
import itertools
import re
import random
import math
import time
from ast import literal_eval
from collections import Counter

from tqdm import tqdm

import multiprocessing
import os, shutil, sys
from pathlib import Path
from glob import glob
from google.colab.patches import cv2_imshow

# packages for processing images
# from PIL import Image
import cv2
import PIL
from skimage import exposure
from skimage.filters import threshold_otsu
from sklearn.cluster import KMeans
from skimage.exposure import is_low_contrast
import skimage.filters as filters
from skimage.feature import local_binary_pattern
from scipy.stats import mode, norm

from progressbar import ProgressBar, Bar, SimpleProgress
from PIL import Image
from PIL import ImageFile
from IPython.display import Image, clear_output
from scipy import ndimage
from skimage.feature import local_binary_pattern
from skimage.feature import hog
ImageFile.LOAD_TRUNCATED_IMAGES = True

from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_score, GridSearchCV, validation_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import (confusion_matrix, r2_score, recall_score,
                             precision_score, f1_score, accuracy_score)
from sklearn.model_selection import cross_val_score, GridSearchCV, validation_curve
from sklearn.pipeline import make_pipeline
from sklearn.utils import class_weight, shuffle
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import plotly.express as px
%matplotlib inline

# tensorflow packages and modules
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow import keras
from tensorflow.keras import Input, layers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (Dense, Conv2D, MaxPooling2D, BatchNormalization, Lambda,
                                     Concatenate, Activation, Dropout, Flatten, Rescaling,
                                     GlobalAveragePooling2D, GlobalMaxPooling2D)
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam
import keras_tuner
from tensorflow.keras.applications import VGG16, InceptionV3, MobileNetV2
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

from tensorflow.python.keras.utils.data_utils import Sequence

from tensorflow.keras.utils import (to_categorical, plot_model,
                                    array_to_img, img_to_array, load_img)

from tensorflow.keras.preprocessing.image import ImageDataGenerator, apply_affine_transform
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau,
                                        LearningRateScheduler, CSVLogger, ModelCheckpoint,
                                        TensorBoard)
from tensorboard import notebook

# Import library for image augmentation
import imgaug
from imgaug import augmenters as iaa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
# Save requirements as text file for reproducibility
# !pip freeze > requirements.txt

In [ ]:
# Find out TensorFlow version you are working on
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.13.0


# Set Seed

In [ ]:
# Set seed for reproducibility in different runtimes
SEED = 42

# (Optional) Import Google Drive Directory if running this notebook on Google Colaboratory

In [ ]:
# Load Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Replace with your designated working directory
%cd /content/drive/MyDrive/uploaded_datasets/

/content/drive/MyDrive/uploaded_datasets


In [ ]:
def create_folder(base_dir, folder_name):
  """
  Creates a folder in designated directory and returns relative path
  """
  new_path = os.path.join(base_dir, folder_name)
  try:
    os.makedirs(new_path)
  except FileExistsError:
    print('Folder exists')

  return new_path

In [ ]:
# Root directory containing project folders and files
DATASET_DIR = os.path.join(os.getcwd(), "pill-classification")
# Dataset Directory (original images)
IMAGE_DIR = os.path.join(DATASET_DIR, "image_dataset")

# Dataset Directory (.npz)
NPZ_DIR = os.path.join(DATASET_DIR, "npz")
# Directories of preprocessed dataset
COLOUR_DIR = os.path.join(DATASET_DIR, "colour_dataset")
TEXTURE_DIR = os.path.join(DATASET_DIR, "texture_dataset")

In [ ]:
# Root directory to store model related outcomes
ROOT_RESULT_DIR = create_folder(DATASET_DIR, 'all-results')

# Directories to store baseline results
BASELINE_CNN_RESULT_DIR = create_folder(ROOT_RESULT_DIR, 'baseline-results-cnn')
BASELINE_CNN_TB_DIR = create_folder(ROOT_RESULT_DIR, 'baseline-tb-cnn')

# Directories to store single-input pre-trained results
# PP_SINGLE_RESULT_DIR = create_folder(ROOT_RESULT_DIR, 'single-results-pp')
# PP_SINGLE_TB_DIR = create_folder(ROOT_RESULT_DIR, 'single-tb-pp')

# Directories to store multi-input pre-trained results
PP_MULTI_RESULT_DIR = create_folder(ROOT_RESULT_DIR, 'multi-results-pp')
PP_MULTI_TB_DIR = create_folder(ROOT_RESULT_DIR, 'multi-tb-pp')

Folder exists
Folder exists
Folder exists


A full dataset including preprocessed images and texture arrays were saved to be reused in this notebook due frequent running out of RAM and compute units.

Hence, you will see instances of loading train, validation, and test sets using .csv and .npz files. As a result, the splitting you observe here does not hold true.

You may download and refer to the following files if you wish to run this notebook containing iterative experimental phase:
- 'train_directory.csv': for train and validation sets
- 'test_directory.csv': for test set

# Exploratory Data Analysis

In [ ]:
# Read directory of filepaths and relevant groundtruths from derived Pillbox CSV file
directory_df = pd.read_csv(os.path.join(DATASET_DIR, 'original_directory.csv'),
                           index_col=0,
                           dtype={
                              'ndc': str,
                              'ndc9': str
                              })
directory_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5449 entries, 0 to 5448
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ndc9           5449 non-null   object
 1   filename       5449 non-null   object
 2   is_ref         5449 non-null   bool  
 3   is_front       5449 non-null   bool  
 4   ndc            5449 non-null   object
 5   splshape       5449 non-null   object
 6   splshape_text  5449 non-null   object
 7   splimprint     5442 non-null   object
 8   splcolor_text  5449 non-null   object
 9   rxstring       4881 non-null   object
 10  filepath       5449 non-null   object
dtypes: bool(2), object(9)
memory usage: 436.3+ KB


## Data Sample Distribution

In [ ]:
# Get class distribution values
class_distrib = directory_df.ndc9.value_counts().to_dict()
class_names = class_distrib.keys()

In [ ]:
# Plot to visualise data distribution across full dataset
fig = px.histogram(directory_df,
                   x="ndc9",
                   title="Class Distribution")
fig.show()

In [ ]:
print("Number of classes: ", len(list(class_distrib.keys())))
print("Total number of samples in dataset: ", len((directory_df)))

Number of classes:  851
Total number of samples in dataset:  5449


## Issue: Sparse dataset and class imbalance

To refine our problem space and minimise predictive errors during the building of a model, we seek to first reduce the dataset by retaining 40 classes that contain the highest number of sample images. Thereafter, we can consider assigning class weights and artificially increasing the number of samples.

In [ ]:
# Plot and visualise distribution of samples in top 40 classes
fig = px.pie(
    names=list(class_names)[:40],
    values=list(class_distrib.values())[:40],
    width=600,
    hole=.3,
    title="Class Distribution"
)
fig.update_layout({'title':{'x': .45}})
fig.show()

In [ ]:
# Retain 40 classes containing highest number of samples
data_df = directory_df[directory_df['ndc9'].isin(list(class_names)[:40])].reset_index(drop=True)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ndc9           556 non-null    object
 1   filename       556 non-null    object
 2   is_ref         556 non-null    bool  
 3   is_front       556 non-null    bool  
 4   ndc            556 non-null    object
 5   splshape       556 non-null    object
 6   splshape_text  556 non-null    object
 7   splimprint     556 non-null    object
 8   splcolor_text  556 non-null    object
 9   rxstring       556 non-null    object
 10  filepath       556 non-null    object
dtypes: bool(2), object(9)
memory usage: 40.3+ KB


Replace *filepath* to your desired path

In [ ]:
# Replace path to images
data_df['filepath'] = [os.path.join(IMAGE_DIR,
                                    filepath.split('/')[-2],
                                    filepath.split('/')[-1]) for filepath in data_df['filepath'].values]

In [ ]:
# Define classes as unique NDC9 values
CLASSES = np.unique(data_df["ndc9"].values)
# Define number of classes
NUM_CLASSES = len(CLASSES)

print("Number of classes in new dataset: ", NUM_CLASSES)

Number of classes in new dataset:  40


# Splitting the Dataset

Due to the imbalanced number of samples in each class, the stratified split method was utilised to preserve the original proportion of samples in each class.

To minimise biasness in the dataset, different class weights were assigned to both majority and minority classes. In laymen terms, this means that higher class weights were set for the minority class, while lowered weights were given to the other.

In [ ]:
# Define ratio of splits
VAL_SPLIT = 0.2
TEST_SPLIT = 0.1

In [ ]:
# # Split dataset into train, test, and validaton sets with stratification
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT,
#                                                   stratify=y, random_state=SEED)

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=VAL_SPLIT,
#                                                   stratify=y_train, random_state=SEED)

In [ ]:
# # Define X and y variables
# # Assign image file paths to X
# X = data_df['filepath'].values
# # Convert NDC9 strings to unique numerical categories
# y_cat = data_df['ndc9'].astype('category').cat.codes.values

# # Observe shapes of X and y
# print(X.shape)
# print(y.shape)

(556,)
(556, 40)


# Image Preprocessing: Final Functions implemented

Applying image pre-processing techniques to enhance or extract key features to learn.

1. Colour (Images)
2. Shape and Texture (HOG-LBP Feature Vectors)

## Colour: Image Preprocessing

In [ ]:
class ColourPreprocessor(object):
  """
  Preproccesses dataset of images for colour enhancement
  """

  def __init__(self, target_size=224, clip_limit=3.0, tile_grid_size=(5, 5)):

    # Initialise and assign variables
    self.target_size = target_size
    self.clip_limit = clip_limit
    self.tile_grid_size = tile_grid_size

  def image_generator(self, img):
    """
    Generates preprocessed image
    """
    try:
      # Preprocess image
      pp_img = self.preprocess_image(img)

      return pp_img

    except Exception as e:
      print("[Image generator] Error occured:\n", e)

  def preprocess_image(self, img):
    """
    Function that calls and applies all image preprocessing techniques
    """
    try:
      # Apply blurring to reduce color inconsistencies and imprints
      img = cv2.GaussianBlur(img, (3, 3), 0)

      # Apply histogram equalisation
      img = self.hist_equalisation(img)

      return img

    except Exception as e:
      print("[Image preprocessing] Error occured:\n", e)

  def hist_equalisation(self, img):
    """
    Function to apply histogram equalisation technique to L channel of LAB image
    """
    try:
      # Convert image to LAB
      lab_img  = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

      # split the image into L, A, B channels
      l_channel, a_channel, b_channel = cv2.split(lab_img)

      # Apply filter to smoothen image
      lab_img = cv2.bilateralFilter(lab_img, 7 , 11, 11)

      # Apply histogram equalisation technique on lightness (L) channel
      clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
      clahed_l = clahe.apply(l_channel)

      # Merge clahed_l with the remaining untouched A, B channels
      merged_channels = cv2.merge((clahed_l, a_channel, b_channel))

      # Convert back to BGR image
      final_output = cv2.cvtColor(merged_channels, cv2.COLOR_LAB2BGR)

      # Return final processed image
      return final_output

    except Exception as e:
      print("[Histogram equalisation] Error occured:\n", e)

In [ ]:
def colour_preprocessing_pipeline(image_paths,
                                 is_float=True,
                                 image_size=IMAGE_SIZE,
                                 clip_limit=3.0, tile_grid_size=(5, 5)):
  """
  Function to process pipeline for colour preprocessing
  """
  #Instantiate class instance
  colour_preprocessor = ColourPreprocessor(target_size=image_size,
                                           clip_limit=clip_limit,
                                           tile_grid_size=tile_grid_size)

  # Store preprocessed data
  X = []

  # Apply functions on each image
  for image_path in tqdm(image_paths):
    # Read image
    input_img = cv2.imread(image_path)
    # Preprocess image
    img = colour_preprocessor.image_generator(input_img)

    # Append data to respective lists
    X.append(img)

  # Return preprocessed data
  if is_float:
    return np.array(X, np.float32)
  return np.array(X)

In [ ]:
# Preprocess datasets
X_train_colour_image = colour_preprocessing_pipeline(image_paths=train_df["original_filepath"].values)
X_val_colour_image = colour_preprocessing_pipeline(image_paths=val_df["original_filepath"].values)
X_test_colour_image = colour_preprocessing_pipeline(image_paths=test_df["original_filepath"].values)

## Shape + Texture Descriptor (Handcrafted features)

Local Binary Pattern (LBP) is widely used in texture analysis to generate more intricate texture-based features. The LBP algorithm computes a new pixel value by  comparing the neighbours of each pixel with its center pixel as a threshold, where the sequence of binary values form the new final pixel in decimal form.

Features of LBP were transformed into a feature histogram to efficiently represent textural properties of both the shape and imprint text regions in each image.

To further improve feature engineering, the integration of Histogram of Oriented Gradients (HOG) was introduced as a descriptor for object localisation. It is a technique that counts events of gradient orientation in specific regions of an image.

These two methods have been long-standing techniques used in facial recogntion models for its effectiveness in detecting key facial structures, in consideration of variations in illumination and occlusions.

Combined, the LBP histogram and HOG feature vectors becomes a single array for each image to be fed as numerical input into a fusion model.

In [ ]:
class TextureDescriptor(object):
  """
  Preproccesses original dataset of images and extracts feature vectors for texture
  """

  def __init__(self, target_size=224):
    # Initialise and assign variables
    self.target_size = target_size

  def get_combined_features(self, img):
    """
    Generates combined feature array
    """
    try:
      # Preprocess image
      pp_img = self.preprocess_image(img)

      # Get feature vector from lbp_extractor
      lbp_features = self.lbp_extractor(pp_img)
      # Get feature vector from hog_extractor
      hog_features = self.hog_extractor(pp_img)

      return np.hstack([lbp_features, hog_features])

    except Exception as e:
      print("[Combined feature generator] Error occured:\n", e)


  def resize_img(self, img):
    """
    Resizes image to target size
    """
    try:
      # https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/
      # (h, w) of original image
      og_size = img.shape[:2]
      # Find ratio to prevent distortion
      ratio = float(self.target_size)/max(og_size)
      # Ratio-ed (h, w) of desired size
      new_size = tuple([int(x*ratio) for x in og_size])
      # (w, h) of target resized image
      img = cv2.resize(img, (new_size[1], new_size[0]))
      # Find differences of desired size (512) and newly computed size
      delta_w = self.target_size - new_size[1]
      delta_h = self.target_size - new_size[0]
      # Compute values for image padding to make black borders
      top, bottom = delta_h//2, delta_h-(delta_h//2)
      left, right = delta_w//2, delta_w-(delta_w//2)
      # Generate final binary image according to target size
      final_img = cv2.copyMakeBorder(img, top, bottom, left, right,
                                    cv2.BORDER_CONSTANT,
                                    value=[0, 0, 0])

      return final_img

    except Exception as e:
      print("[Image resizing] Error occured:\n", e)

  def lbp_extractor(self, img):
    """
    Returns derived LBP bins as array
    """
    # Define values for lbp
    radius= 2
    num_pts = 10
    bin = 2**num_pts

    # Compute lbp
    lbp = local_binary_pattern(img, num_pts, radius, method='uniform')
    # Compute lbp histograms by bins
    (hist, hist_len) = np.histogram(lbp.ravel(), bins=np.arange(0, bin))
    lbp_hist = hist.astype("float32")

    # Return array in float32 format
    return np.array(lbp_hist, np.float32)

  def hog_extractor(self, img):
    """
    Returns derived HOG array
    """
    fd, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2),
                        visualize=True, channel_axis=None)

    # Rescale histogram for better display
    fd = exposure.rescale_intensity(fd, in_range=(0, 10))

    # Return array in float32 format
    return np.array(fd, np.float32)

  def preprocess_image(self, img):
    """
    Image preprocessing before extracting shape and textural features
    """
    try:
      # Make sure all images have the same size
      img = self.resize_img(img)

      # Convert image to greyscale
      grey_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

      # Apply gaussian filter
      gaussian = cv2.GaussianBlur(grey_img, (3, 3) ,cv2.BORDER_DEFAULT)

      return grey_img

    except Exception as e:
      print("[Image Preprocessing] Error occured:\n", e)

In [ ]:
def texture_preprocessing_pipeline(image_paths,
                                   image_size=IMAGE_SIZE):
  """
  Function to process pipeline for texture feature extracton
  """
  #Instantiate class instance
  texture_extractor = TextureDescriptor(target_size=image_size)

  # Store preprocessed data
  X = []

  # Apply functions on each image and assign respective labels
  for image_path in tqdm(image_paths):
    # Read image
    input_img = cv2.imread(image_path)
    # Get feature vector
    features = texture_extractor.get_combined_features(input_img)

    # Append data to respective lists
    X.append(features)

  # Return preprocessed set of data
  return np.array(X)

Load final split dataset containing pre-processed images

In [ ]:
# Load datasets from .npz files
train_data = np.load(os.path.join(NPZ_DIR, 'full_trainset.npz'), allow_pickle=True)
test_data = np.load(os.path.join(NPZ_DIR, 'full_testset.npz'), allow_pickle=True)

In [ ]:
# Dictionaries to store datasets
train_set = dict()
test_set = dict()

# Unload objects from .npy files into dictionary of key: np.array() items
for key in train_data.files:
  train_set[key] = train_data[key]
for key in test_data.files:
  test_set[key] = test_data[key]

# Distinguish between train and validation sets
val_set = {k: train_set[k][np.where(train_set['is_train']==False)] for k in train_set.keys()}
train_set = {k: train_set[k][np.where(train_set['is_train']==True)] for k in train_set.keys()}

Form dataframes for single-input model's ImageDataGenerator

In [ ]:
train_df = pd.DataFrame.from_dict({key: value for key, value in train_set.items() if key in ['ndc9', 'original_filepath', 'colour_filepath'] })
val_df = pd.DataFrame.from_dict({key: value for key, value in val_set.items() if key in ['ndc9', 'original_filepath', 'colour_filepath'] })
test_df = pd.DataFrame.from_dict({key: value for key, value in test_set.items() if key in ['ndc9', 'original_filepath', 'colour_filepath'] })

In [ ]:
# Replace absolute path with relative path
for col in ["original_filepath", "colour_filepath"]:
  train_df[col] = train_df[col].apply(lambda x: f"{os.getcwd()}/{DATASET_DIR}/{x}")
  val_df[col] = val_df[col].apply(lambda x: f"{os.getcwd()}/{DATASET_DIR}/{x}")
  test_df[col] = test_df[col].apply(lambda x: f"{os.getcwd()}/{DATASET_DIR}/{x}")

Form dictionaries with keys as image paths for custom data generator class

In [ ]:
def filepath_data_dict(filepath_array, data_array, get_relative_path=True):
  """
  Returns a dictionary with filepaths as keys for data generator
  """
  # Store in a dictionary
  output_dict = dict()

  for filepath, data in zip(filepath_array, data_array):
    if get_relative_path:
      filepath = f"{os.getcwd()}/{DATASET_DIR}/{filepath}"

    output_dict[filepath] = data

  return output_dict

def create_folder(file_path):
  try:
    os.makedirs(file_path)
  except FileExistsError:
    print('Folder exists')

In [ ]:
# Generate respective dictionaries
## X_TRAIN
X_train_dict = filepath_data_dict(train_set['colour_filepath'],
                                  train_set['feature'])

## X_VAL
X_val_dict = filepath_data_dict(val_set['colour_filepath'],
                                val_set['feature'])

## X_TEST
X_test_dict = filepath_data_dict(test_set['colour_filepath'],
                                 test_set['feature'])

## Y_TRAIN
y_train_dict = filepath_data_dict(train_set['colour_filepath'],
                                  train_set['y_cat'])

## Y_VAL
y_val_dict = filepath_data_dict(val_set['colour_filepath'],
                                val_set['y_cat'])

## Y_TEST
y_test_dict = filepath_data_dict(test_set['colour_filepath'],
                                 test_set['y_cat'])

In [ ]:
print("All keys in X and y sets have to be identical: ")
print("Train set", set(X_train_dict.keys()) == set(train_df["colour_filepath"].values))
print("Val set", set(X_val_dict.keys()) == set(val_df["colour_filepath"].values))
print("Test test", set(X_test_dict.keys()) == set(test_df["colour_filepath"].values))

All keys in X and y sets have to be identical: 
Train set True
Val set True
Test test True


In [ ]:
print("All keys in X and y sets have to be identical: ")
print("Train set", X_train_dict.keys() == y_train_dict.keys())
print("Val set", X_val_dict.keys() == y_val_dict.keys())
print("Test test", X_test_dict.keys() == y_test_dict.keys())

All keys in X and y sets have to be identical: 
Train set True
Val set True
Test test True


# Overview of Functions for Training Models

## Common Training Functions

- `add_callbacks`: Adds callbacks to customise behaviour of model
- `get_class_weights`: Computes class weights for sample balance
- `compile_model`: Compiles model
- `save_model_archi`: Saves model architecture as image with filename and model as inputs

## Functions to train baseline mode

- `add_callbacks`
- `get_class_weights`
- `compile_model`
- `build_colour_baseline`: Builds and returns baseline CNN model
- `train_baseline_model`: Trains model with inputs from ImageDataGenerator


# Training

## Common Functions

- add_callbacks
- get_class_weights
- save_model_archi
- compile_model

Tensorboard

In [ ]:
# Load tensorboard to view the network
%load_ext tensorboard

In [ ]:
def add_callbacks(patience):
  """
  Returns additional callbacks for model,
  arg: patience value
  """
  lr_reduce = ReduceLROnPlateau(monitor='val_loss', mode="min",
                                factor=0.2, min_lr=1e-5,
                                patience=patience, verbose=1),
  early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=patience)

  return [lr_reduce, early_stopping]


def get_class_weights(y_train, is_encoded=False):
  """
  Computes class weights to resolve imbalanced dataset
  """

  #Cconvert from np.array of one-hot encoded labels to integers
  if is_encoded:
    y_train_encoded = y_train
    y_train = [np.argmax(lbl) for lbl in y_train_encoded]

  # Compute class weights
  class_weights = class_weight.compute_class_weight(
      'balanced', classes=np.unique(y_train), y=y_train)

  # Assign weights for model
  class_weights_dict = dict(enumerate(class_weights))

  return class_weights_dict

def save_model_archi(model, filename):
  tf.keras.utils.plot_model(
      model,
      to_file="{}.png".format(filename),
      show_shapes=True,
      show_layer_names=True,
      show_layer_activations=True
  )

  print(f"Saved to {filename}")

## Function to compile model

In [ ]:
def compile_model(model,
                  optimizer,
                  loss='categorical_crossentropy'):
    """
    Compiles a model with selected metrics,
    args: optimizer, loss
    """
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=[
            'accuracy',
            tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top_k_categorical_accuracy'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.FalsePositives(name='false_positives')
        ]
    )

    return model

# (1) Baseline model: Single-input CNN model from Scratch

## Training Functions

In [ ]:
def build_colour_baseline(model_name,
                          num_classes=NUM_CLASSES,
                          img_size=IMAGE_SIZE):

  """
  Builds and returns baseline CNN model for Colour stream
  """

  print(model_name)

  # Create a Sequential model
  model= Sequential()
  # Define inputs
  input_image = Input(shape=(img_size, img_size, 3), name="coloured_image")

  # Stack conv layers
  model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # Create fully connected layers
  model.add(Flatten())

  # Add Dense layer
  model.add(Dense(128, activation='relu'))
  # Add a Dropout layer
  model.add(Dropout(0.25))

  # Add Dense layer
  model.add(Dense(64, activation='relu'))
  # Add a Dropout layer
  model.add(Dropout(0.25))

  model.add(Dense(num_classes, activation='softmax', name='predictions'))

  # Print model summary
  print(model.summary())

  return model

def train_model(
  model, model_name,
  train_gen, val_gen,
  X_train,
  X_val,
  batch_size,
  class_weights,
  result_dir,
  log_dir,
  version,
  workers, use_multiprocessing,
  epochs=20,
  additional_callbacks=[]):
  """
  Trains model with inputs from ImageDataGenerator
  """

  if not os.path.exists(os.path.join(result_dir, model_name)):
      os.makedirs(os.path.join(result_dir, model_name))

  if not os.path.exists(os.path.join(log_dir, model_name)):
      os.makedirs(os.path.join(log_dir, model_name))

  model_log_dir = os.path.join(log_dir, model_name, f'{model_name}_{version}')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(model_log_dir, histogram_freq=1)

  history = model.fit(
      train_gen,
      steps_per_epoch=train_gen.n // train_gen.batch_size,
      validation_data=val_gen,
      validation_steps=val_gen.n // val_gen.batch_size,
      class_weight=class_weights,
      epochs=epochs,
      workers=workers,
      use_multiprocessing=use_multiprocessing,
      verbose=1,
      callbacks=[
          tensorboard_callback,
          tf.keras.callbacks.TerminateOnNaN(),
          tf.keras.callbacks.CSVLogger(os.path.join(result_dir, model_name, f'{model_name}_{version}.log'), separator=',', append=True),
          tf.keras.callbacks.ModelCheckpoint(os.path.join(result_dir, model_name, f'{model_name}_{version}'),
                                monitor='val_loss', verbose=1,
                                save_best_only=True, mode='min')
      ] + additional_callbacks
  )

  return history, model

## Image Data Generator

In [ ]:
# TRAIN
base_train_datagen = ImageDataGenerator(rescale=1./255)

base_train_generator = base_train_datagen.flow_from_dataframe(
    train_df,
    x_col='colour_filepath',
    y_col='ndc9',
    shuffle=True,
    seed=SEED,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    class_mode='categorical'
)

# VAL and TEST
base_test_datagen = ImageDataGenerator(rescale=1./255)

base_val_generator = base_test_datagen.flow_from_dataframe(
    val_df,
    x_col='colour_filepath',
    y_col='ndc9',
    shuffle=False,
    seed=SEED,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    class_mode='categorical'
)

base_test_generator = base_test_datagen.flow_from_dataframe(
    test_df,
    x_col='colour_filepath',
    y_col='ndc9',
    shuffle=False,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    class_mode='categorical'
)

Found 400 validated image filenames belonging to 40 classes.
Found 100 validated image filenames belonging to 40 classes.
Found 56 validated image filenames belonging to 40 classes.


In [ ]:
# Reset generator if needed
base_train_generator.reset()
base_val_generator.reset()

## Build and Train Model

In [ ]:
# Name of model to build
model_name = "Single_Baseline_PP"

# Build baseline model
single_baseline_model = build_colour_baseline(model_name)

Single_Baseline_PP
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 

In [ ]:
# Compile model
single_baseline_model = compile_model(single_baseline_model, optimizer=Adam())

# Train model
single_baseline_model_history, single_baseline_model = train_model(
    single_baseline_model, model_name,
    train_gen=base_train_generator,
    val_gen=base_val_generator,
    X_train=train_df['colour_filepath'].values,
    X_val=val_df['colour_filepath'].values,
    class_weights=get_class_weights(base_train_generator.classes),
    result_dir=BASELINE_CNN_RESULT_DIR,
    log_dir=BASELINE_CNN_TB_DIR,
    additional_callbacks=add_callbacks(4),
    epochs=50,
    workers=2,
    use_multiprocessing=False,
    version='coloured_base_v1',
    batch_size=BATCH_SIZE)

Epoch 1/50
12/12 [==============================] - ETA: 0s - loss: 3.7758 - accuracy: 0.0217 - top_k_categorical_accuracy: 0.1141 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 1: val_loss improved from inf to 3.66672, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/baseline-results-cnn/Single_Baseline_PP/Single_Baseline_PP_coloured_base_v1


12/12 [==============================] - 10s 852ms/step - loss: 3.7758 - accuracy: 0.0217 - top_k_categorical_accuracy: 0.1141 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.6667 - val_accuracy: 0.1146 - val_top_k_categorical_accuracy: 0.2812 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 2/50
11/12 [==========================>...] - ETA: 0s - loss: 3.6266 - accuracy: 0.0653 - top_k_categorical_accuracy: 0.1932 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 2: val_loss improved from 3.66672 to 3.55154, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/baseline-results-cnn/Single_Baseline_PP/Single_Baseline_PP_coloured_base_v1


12/12 [==============================] - 9s 784ms/step - loss: 3.6279 - accuracy: 0.0625 - top_k_categorical_accuracy: 0.1957 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.5515 - val_accuracy: 0.1250 - val_top_k_categorical_accuracy: 0.3333 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 3/50
11/12 [==========================>...] - ETA: 0s - loss: 3.4879 - accuracy: 0.0625 - top_k_categorical_accuracy: 0.2589 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 3: val_loss improved from 3.55154 to 3.33199, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/baseline-results-cnn/Single_Baseline_PP/Single_Baseline_PP_coloured_base_v1


12/12 [==============================] - 9s 830ms/step - loss: 3.4919 - accuracy: 0.0625 - top_k_categorical_accuracy: 0.2582 - precision: 1.0000 - recall: 0.0027 - false_positives: 0.0000e+00 - val_loss: 3.3320 - val_accuracy: 0.1667 - val_top_k_categorical_accuracy: 0.3958 - val_precision: 1.0000 - val_recall: 0.0208 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 4/50
12/12 [==============================] - ETA: 0s - loss: 3.3374 - accuracy: 0.1332 - top_k_categorical_accuracy: 0.3424 - precision: 0.7778 - recall: 0.0190 - false_positives: 2.0000
Epoch 4: val_loss improved from 3.33199 to 3.11247, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/baseline-results-cnn/Single_Baseline_PP/Single_Baseline_PP_coloured_base_v1


12/12 [==============================] - 9s 770ms/step - loss: 3.3374 - accuracy: 0.1332 - top_k_categorical_accuracy: 0.3424 - precision: 0.7778 - recall: 0.0190 - false_positives: 2.0000 - val_loss: 3.1125 - val_accuracy: 0.2083 - val_top_k_categorical_accuracy: 0.4271 - val_precision: 0.7500 - val_recall: 0.0625 - val_false_positives: 2.0000 - lr: 0.0010
Epoch 5/50
11/12 [==========================>...] - ETA: 0s - loss: 3.2262 - accuracy: 0.1429 - top_k_categorical_accuracy: 0.3958 - precision: 0.5000 - recall: 0.0179 - false_positives: 6.0000
Epoch 5: val_loss improved from 3.11247 to 3.01006, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/baseline-results-cnn/Single_Baseline_PP/Single_Baseline_PP_coloured_base_v1


...

12/12 [==============================] - 9s 807ms/step - loss: 1.9264 - accuracy: 0.4049 - top_k_categorical_accuracy: 0.8098 - precision: 0.7049 - recall: 0.2337 - false_positives: 36.0000 - val_loss: 2.1521 - val_accuracy: 0.4062 - val_top_k_categorical_accuracy: 0.7083 - val_precision: 0.7778 - val_recall: 0.2188 - val_false_positives: 6.0000 - lr: 0.0010
Epoch 15/50
12/12 [==============================] - ETA: 0s - loss: 1.7713 - accuracy: 0.4484 - top_k_categorical_accuracy: 0.7772 - precision: 0.7537 - recall: 0.2745 - false_positives: 33.0000
Epoch 15: val_loss did not improve from 2.15210
12/12 [==============================] - 7s 647ms/step - loss: 1.7713 - accuracy: 0.4484 - top_k_categorical_accuracy: 0.7772 - precision: 0.7537 - recall: 0.2745 - false_positives: 33.0000 - val_loss: 2.2644 - val_accuracy: 0.3542 - val_top_k_categorical_accuracy: 0.6979 - val_precision: 0.6774 - val_recall: 0.2188 - val_false_positives: 10.0000 - lr: 0.0010
Epoch 16/50
12/12 [==============

12/12 [==============================] - 9s 780ms/step - loss: 1.8397 - accuracy: 0.4511 - top_k_categorical_accuracy: 0.7826 - precision: 0.7364 - recall: 0.2582 - false_positives: 34.0000 - val_loss: 2.0917 - val_accuracy: 0.4062 - val_top_k_categorical_accuracy: 0.7083 - val_precision: 0.7917 - val_recall: 0.1979 - val_false_positives: 5.0000 - lr: 0.0010
Epoch 17/50
12/12 [==============================] - ETA: 0s - loss: 1.5720 - accuracy: 0.5190 - top_k_categorical_accuracy: 0.8125 - precision: 0.7812 - recall: 0.3397 - false_positives: 35.0000
Epoch 17: val_loss did not improve from 2.09172
12/12 [==============================] - 7s 620ms/step - loss: 1.5720 - accuracy: 0.5190 - top_k_categorical_accuracy: 0.8125 - precision: 0.7812 - recall: 0.3397 - false_positives: 35.0000 - val_loss: 2.0947 - val_accuracy: 0.4062 - val_top_k_categorical_accuracy: 0.7396 - val_precision: 0.7419 - val_recall: 0.2396 - val_false_positives: 8.0000 - lr: 0.0010
Epoch 18/50
12/12 [===============

12/12 [==============================] - 9s 761ms/step - loss: 1.4634 - accuracy: 0.5109 - top_k_categorical_accuracy: 0.8723 - precision: 0.7471 - recall: 0.3533 - false_positives: 44.0000 - val_loss: 2.0538 - val_accuracy: 0.4271 - val_top_k_categorical_accuracy: 0.7396 - val_precision: 0.6970 - val_recall: 0.2396 - val_false_positives: 10.0000 - lr: 0.0010
Epoch 19/50
11/12 [==========================>...] - ETA: 0s - loss: 1.5230 - accuracy: 0.5060 - top_k_categorical_accuracy: 0.8631 - precision: 0.7200 - recall: 0.3750 - false_positives: 49.0000
Epoch 19: val_loss did not improve from 2.05377
12/12 [==============================] - 7s 622ms/step - loss: 1.5034 - accuracy: 0.5245 - top_k_categorical_accuracy: 0.8641 - precision: 0.7382 - recall: 0.3832 - false_positives: 50.0000 - val_loss: 2.0867 - val_accuracy: 0.4062 - val_top_k_categorical_accuracy: 0.7396 - val_precision: 0.6250 - val_recall: 0.2604 - val_false_positives: 15.0000 - lr: 0.0010
Epoch 20/50
12/12 [=============

12/12 [==============================] - 9s 794ms/step - loss: 1.3455 - accuracy: 0.5842 - top_k_categorical_accuracy: 0.8859 - precision: 0.7680 - recall: 0.4049 - false_positives: 45.0000 - val_loss: 2.0186 - val_accuracy: 0.4375 - val_top_k_categorical_accuracy: 0.7812 - val_precision: 0.6818 - val_recall: 0.3125 - val_false_positives: 14.0000 - lr: 0.0010
Epoch 21/50
12/12 [==============================] - ETA: 0s - loss: 1.2378 - accuracy: 0.6060 - top_k_categorical_accuracy: 0.9076 - precision: 0.8086 - recall: 0.4592 - false_positives: 40.0000
Epoch 21: val_loss improved from 2.01862 to 1.98956, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/baseline-results-cnn/Single_Baseline_PP/Single_Baseline_PP_coloured_base_v1


12/12 [==============================] - 9s 781ms/step - loss: 1.2378 - accuracy: 0.6060 - top_k_categorical_accuracy: 0.9076 - precision: 0.8086 - recall: 0.4592 - false_positives: 40.0000 - val_loss: 1.9896 - val_accuracy: 0.4479 - val_top_k_categorical_accuracy: 0.7604 - val_precision: 0.6923 - val_recall: 0.2812 - val_false_positives: 12.0000 - lr: 0.0010
Epoch 22/50
11/12 [==========================>...] - ETA: 0s - loss: 1.2188 - accuracy: 0.5714 - top_k_categorical_accuracy: 0.9167 - precision: 0.8261 - recall: 0.4524 - false_positives: 32.0000
Epoch 22: val_loss did not improve from 1.98956
12/12 [==============================] - 7s 613ms/step - loss: 1.1939 - accuracy: 0.5870 - top_k_categorical_accuracy: 0.9239 - precision: 0.8284 - recall: 0.4592 - false_positives: 35.0000 - val_loss: 2.1281 - val_accuracy: 0.4167 - val_top_k_categorical_accuracy: 0.7292 - val_precision: 0.6296 - val_recall: 0.3542 - val_false_positives: 20.0000 - lr: 0.0010
Epoch 23/50
12/12 [=============

# (2) Single-input with Transfer Learning & Augmentation


## Training Functions

In [ ]:
def build_cnn_model(base_model, model_name, is_trainable,
                    drop_out=0.2,
                    num_classes=NUM_CLASSES, img_size=IMAGE_SIZE):

  print(model_name)

  # Freeze convolutional base
  if is_trainable == False:
    base_model.trainable = False

  # Define image inputs
  inputs = tf.keras.Input(shape=(img_size, img_size, 3), name="coloured_image")

  # Instantiate pre-trained model
  if 'MobileNet' in model_name:
    # Pre-process input for model
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)

  # Instantiate pre-trained model
  elif 'ResNet50' in model_name:
    # Pre-process input for model
    x = tf.keras.applications.resnet_v2.preprocess_input(inputs)

  else:
    print('Backbone not found')

  x = base_model(x, training=False)
  # Add a classification layers
  x = GlobalAveragePooling2D()(x)
  # Add a Dropout layer
  x = Dropout(drop_out)(x)
  # Add Dense layers
  x = Dense(256, activation='relu')(x)
  x = Dense(128, activation='relu')(x)
  # Add Dense layer to output predictions
  predictions = Dense(num_classes, activation='softmax', name='predictions')(x)

  # Create final model
  model = tf.keras.Model(inputs=inputs, outputs=predictions)
  # Print model summary
  print(model.summary())

  return model

## Training: MobileNet V2

In [ ]:
# Clear existing sessions
# tf.keras.backend.clear_session()

In [ ]:
# Reset generator
base_train_generator.reset()
base_val_generator.reset()

In [ ]:
model_name = 'MobileNet_Single_PP'

# Load MobileNet V2 with ImageNet weights without last fully connected layers
mobilenet_base_model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                                      weights='imagenet',
                                      include_top=False)

mobilenet_single_pp = build_cnn_model(mobilenet_base_model, model_name,
                             is_trainable=False)

MobileNet_Single_PP
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 coloured_image (InputLayer)  [(None, 224, 224, 3)]    0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                  

In [ ]:
# Compile model
mobilenet_single_pp = compile_model(mobilenet_single_pp, optimizer=Adam())

# Train model
mobilenet_single_pp_history, mobilenet_single_pp = train_model(
    mobilenet_single_pp, model_name,
    train_gen=base_train_generator,
    val_gen=base_val_generator,
    X_train=train_df['colour_filepath'].values,
    X_val=val_df['colour_filepath'].values,
    class_weights=get_class_weights(base_train_generator.classes),
    result_dir=PP_SINGLE_RESULT_DIR,
    log_dir=PP_SINGLE_TB_DIR,
    additional_callbacks=add_callbacks(4),
    epochs=50,
    workers=2,
    use_multiprocessing=False,
    version='v1',
    batch_size=BATCH_SIZE)

Epoch 1/50
12/12 [==============================] - ETA: 0s - loss: 3.8333 - accuracy: 0.0272 - top_k_categorical_accuracy: 0.0951 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 1: val_loss improved from inf to 3.71806, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/MobileNet_Single_PP/MobileNet_Single_PP_v1


12/12 [==============================] - 35s 3s/step - loss: 3.8333 - accuracy: 0.0272 - top_k_categorical_accuracy: 0.0951 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.7181 - val_accuracy: 0.0208 - val_top_k_categorical_accuracy: 0.1354 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 2/50
11/12 [==========================>...] - ETA: 0s - loss: 3.7372 - accuracy: 0.0149 - top_k_categorical_accuracy: 0.1071 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 2: val_loss improved from 3.71806 to 3.70090, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/MobileNet_Single_PP/MobileNet_Single_PP_v1


12/12 [==============================] - 32s 3s/step - loss: 3.7376 - accuracy: 0.0136 - top_k_categorical_accuracy: 0.1005 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.7009 - val_accuracy: 0.0312 - val_top_k_categorical_accuracy: 0.1146 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 3/50
11/12 [==========================>...] - ETA: 0s - loss: 3.7427 - accuracy: 0.0341 - top_k_categorical_accuracy: 0.0966 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 3: val_loss did not improve from 3.70090
12/12 [==============================] - 8s 729ms/step - loss: 3.7375 - accuracy: 0.0312 - top_k_categorical_accuracy: 0.0911 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.7033 - val_accuracy: 0.0208 - val_top_k_categorical_accuracy: 0.1354 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.000

12/12 [==============================] - 30s 3s/step - loss: 3.7113 - accuracy: 0.0190 - top_k_categorical_accuracy: 0.1141 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.6951 - val_accuracy: 0.0208 - val_top_k_categorical_accuracy: 0.1250 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 5/50
11/12 [==========================>...] - ETA: 0s - loss: 3.7023 - accuracy: 0.0208 - top_k_categorical_accuracy: 0.0982 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 5: val_loss did not improve from 3.69515
12/12 [==============================] - 4s 334ms/step - loss: 3.7038 - accuracy: 0.0190 - top_k_categorical_accuracy: 0.0951 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.7021 - val_accuracy: 0.0312 - val_top_k_categorical_accuracy: 0.1354 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.000

12/12 [==============================] - 32s 3s/step - loss: 3.7157 - accuracy: 0.0245 - top_k_categorical_accuracy: 0.1223 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.6950 - val_accuracy: 0.0312 - val_top_k_categorical_accuracy: 0.1250 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 7/50
11/12 [==========================>...] - ETA: 0s - loss: 3.7084 - accuracy: 0.0268 - top_k_categorical_accuracy: 0.0952 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 7: val_loss improved from 3.69498 to 3.69480, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/MobileNet_Single_PP/MobileNet_Single_PP_v1


12/12 [==============================] - 31s 3s/step - loss: 3.7066 - accuracy: 0.0245 - top_k_categorical_accuracy: 0.0951 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.6948 - val_accuracy: 0.0312 - val_top_k_categorical_accuracy: 0.1146 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 8/50
11/12 [==========================>...] - ETA: 0s - loss: 3.6859 - accuracy: 0.0199 - top_k_categorical_accuracy: 0.1364 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 8: val_loss improved from 3.69480 to 3.69313, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/MobileNet_Single_PP/MobileNet_Single_PP_v1


12/12 [==============================] - 31s 3s/step - loss: 3.6930 - accuracy: 0.0182 - top_k_categorical_accuracy: 0.1302 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.6931 - val_accuracy: 0.0312 - val_top_k_categorical_accuracy: 0.1458 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 9/50
12/12 [==============================] - ETA: 0s - loss: 3.6885 - accuracy: 0.0272 - top_k_categorical_accuracy: 0.1332 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 9: val_loss improved from 3.69313 to 3.68361, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/MobileNet_Single_PP/MobileNet_Single_PP_v1


12/12 [==============================] - 31s 3s/step - loss: 3.6885 - accuracy: 0.0272 - top_k_categorical_accuracy: 0.1332 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.6836 - val_accuracy: 0.0312 - val_top_k_categorical_accuracy: 0.1562 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 10/50
12/12 [==============================] - ETA: 0s - loss: 3.6954 - accuracy: 0.0217 - top_k_categorical_accuracy: 0.1141 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 10: val_loss did not improve from 3.68361
12/12 [==============================] - 8s 743ms/step - loss: 3.6954 - accuracy: 0.0217 - top_k_categorical_accuracy: 0.1141 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.7025 - val_accuracy: 0.0312 - val_top_k_categorical_accuracy: 0.1146 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0

## Training: ResNet-50 V2


In [ ]:
# Clear existing sessions if needed
# tf.keras.backend.clear_session()

In [ ]:
# Reset generator
base_train_generator.reset()
base_val_generator.reset()

In [ ]:
model_name = 'ResNet50_Single_PP'

# Load ResNet50 V2 with ImageNet weights without last fully connected layers
resnet50_base_model = ResNet50V2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                                      weights='imagenet',
                                      include_top=False)

resnet50_single_pp = build_cnn_model(resnet50_base_model, model_name,
                                     is_trainable=False)

94668760/94668760 [==============================] - 1s 0us/step
ResNet50_Single_PP
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 coloured_image (InputLayer)  [(None, 224, 224, 3)]    0         
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_1 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                

In [ ]:
# Compile model
resnet50_single_pp = compile_model(resnet50_single_pp, optimizer=Adam())

# Train model
resnet50_single_pp_history, resnet50_single_pp = train_model(
    resnet50_single_pp, model_name,
    train_gen=base_train_generator,
    val_gen=base_val_generator,
    X_train=train_df['colour_filepath'].values,
    X_val=val_df['colour_filepath'].values,
    class_weights=get_class_weights(base_train_generator.classes),
    result_dir=PP_SINGLE_RESULT_DIR,
    log_dir=PP_SINGLE_TB_DIR,
    additional_callbacks=add_callbacks(4),
    epochs=50,
    workers=2,
    use_multiprocessing=False,
    version='v1',
    batch_size=BATCH_SIZE)

Epoch 1/50
12/12 [==============================] - ETA: 0s - loss: 3.8030 - accuracy: 0.0380 - top_k_categorical_accuracy: 0.1141 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 1: val_loss improved from inf to 3.68472, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/ResNet50_Single_PP/ResNet50_Single_PP_v1


12/12 [==============================] - 39s 3s/step - loss: 3.8030 - accuracy: 0.0380 - top_k_categorical_accuracy: 0.1141 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.6847 - val_accuracy: 0.0312 - val_top_k_categorical_accuracy: 0.1875 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 2/50
12/12 [==============================] - ETA: 0s - loss: 3.6842 - accuracy: 0.0380 - top_k_categorical_accuracy: 0.1712 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 2: val_loss improved from 3.68472 to 3.65060, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/ResNet50_Single_PP/ResNet50_Single_PP_v1


12/12 [==============================] - 34s 3s/step - loss: 3.6842 - accuracy: 0.0380 - top_k_categorical_accuracy: 0.1712 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.6506 - val_accuracy: 0.0312 - val_top_k_categorical_accuracy: 0.1979 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 3/50
12/12 [==============================] - ETA: 0s - loss: 3.6364 - accuracy: 0.0462 - top_k_categorical_accuracy: 0.2092 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 3: val_loss improved from 3.65060 to 3.63268, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/ResNet50_Single_PP/ResNet50_Single_PP_v1


12/12 [==============================] - 36s 3s/step - loss: 3.6364 - accuracy: 0.0462 - top_k_categorical_accuracy: 0.2092 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.6327 - val_accuracy: 0.0312 - val_top_k_categorical_accuracy: 0.1979 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 4/50
12/12 [==============================] - ETA: 0s - loss: 3.6178 - accuracy: 0.0462 - top_k_categorical_accuracy: 0.2120 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 4: val_loss improved from 3.63268 to 3.57354, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/ResNet50_Single_PP/ResNet50_Single_PP_v1


12/12 [==============================] - 31s 3s/step - loss: 3.6178 - accuracy: 0.0462 - top_k_categorical_accuracy: 0.2120 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.5735 - val_accuracy: 0.0625 - val_top_k_categorical_accuracy: 0.3125 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 5/50
12/12 [==============================] - ETA: 0s - loss: 3.5709 - accuracy: 0.0734 - top_k_categorical_accuracy: 0.2337 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 5: val_loss improved from 3.57354 to 3.50925, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/ResNet50_Single_PP/ResNet50_Single_PP_v1


...

12/12 [==============================] - 31s 3s/step - loss: 1.9316 - accuracy: 0.4158 - top_k_categorical_accuracy: 0.7962 - precision: 0.8364 - recall: 0.1250 - false_positives: 9.0000 - val_loss: 2.4915 - val_accuracy: 0.2500 - val_top_k_categorical_accuracy: 0.6771 - val_precision: 0.8571 - val_recall: 0.0625 - val_false_positives: 1.0000 - lr: 0.0010
Epoch 46/50
12/12 [==============================] - ETA: 0s - loss: 1.9244 - accuracy: 0.3995 - top_k_categorical_accuracy: 0.7391 - precision: 0.9508 - recall: 0.1576 - false_positives: 3.0000
Epoch 46: val_loss did not improve from 2.49148
12/12 [==============================] - 7s 624ms/step - loss: 1.9244 - accuracy: 0.3995 - top_k_categorical_accuracy: 0.7391 - precision: 0.9508 - recall: 0.1576 - false_positives: 3.0000 - val_loss: 2.4969 - val_accuracy: 0.2917 - val_top_k_categorical_accuracy: 0.6250 - val_precision: 0.7000 - val_recall: 0.0729 - val_false_positives: 3.0000 - lr: 0.0010
Epoch 47/50
12/12 [====================

12/12 [==============================] - 30s 3s/step - loss: 1.8782 - accuracy: 0.4049 - top_k_categorical_accuracy: 0.7880 - precision: 0.8841 - recall: 0.1658 - false_positives: 8.0000 - val_loss: 2.4866 - val_accuracy: 0.2292 - val_top_k_categorical_accuracy: 0.6250 - val_precision: 0.7778 - val_recall: 0.0729 - val_false_positives: 2.0000 - lr: 0.0010
Epoch 48/50
12/12 [==============================] - ETA: 0s - loss: 1.8140 - accuracy: 0.4429 - top_k_categorical_accuracy: 0.8043 - precision: 0.9403 - recall: 0.1712 - false_positives: 4.0000
Epoch 48: val_loss improved from 2.48661 to 2.48005, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/ResNet50_Single_PP/ResNet50_Single_PP_v1


12/12 [==============================] - 32s 3s/step - loss: 1.8140 - accuracy: 0.4429 - top_k_categorical_accuracy: 0.8043 - precision: 0.9403 - recall: 0.1712 - false_positives: 4.0000 - val_loss: 2.4800 - val_accuracy: 0.2812 - val_top_k_categorical_accuracy: 0.6354 - val_precision: 0.8000 - val_recall: 0.0833 - val_false_positives: 2.0000 - lr: 0.0010
Epoch 49/50
12/12 [==============================] - ETA: 0s - loss: 1.8157 - accuracy: 0.4212 - top_k_categorical_accuracy: 0.7989 - precision: 0.8148 - recall: 0.1793 - false_positives: 15.0000
Epoch 49: val_loss improved from 2.48005 to 2.47921, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/ResNet50_Single_PP/ResNet50_Single_PP_v1


12/12 [==============================] - 32s 3s/step - loss: 1.8157 - accuracy: 0.4212 - top_k_categorical_accuracy: 0.7989 - precision: 0.8148 - recall: 0.1793 - false_positives: 15.0000 - val_loss: 2.4792 - val_accuracy: 0.3021 - val_top_k_categorical_accuracy: 0.6875 - val_precision: 0.6250 - val_recall: 0.0521 - val_false_positives: 3.0000 - lr: 0.0010
Epoch 50/50
12/12 [==============================] - ETA: 0s - loss: 1.8044 - accuracy: 0.4647 - top_k_categorical_accuracy: 0.8098 - precision: 0.8732 - recall: 0.1685 - false_positives: 9.0000
Epoch 50: val_loss improved from 2.47921 to 2.46342, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/single-results-pp/ResNet50_Single_PP/ResNet50_Single_PP_v1


12/12 [==============================] - 33s 3s/step - loss: 1.8044 - accuracy: 0.4647 - top_k_categorical_accuracy: 0.8098 - precision: 0.8732 - recall: 0.1685 - false_positives: 9.0000 - val_loss: 2.4634 - val_accuracy: 0.2396 - val_top_k_categorical_accuracy: 0.6562 - val_precision: 0.7692 - val_recall: 0.1042 - val_false_positives: 3.0000 - lr: 0.0010


# (3) Multi-input with Transfer Learning, Augmentation, & Descriptors

Two inputs:
1. Preprocessed image for Colour stream
2. Feature vector for Texture stream

## Custom Mixed-type Data Generator

In [ ]:
class CustomDataGenerator(tf.keras.utils.Sequence):

  def __init__(self,
               image_filepaths, feature_vectors, labels,
               batch_size=BATCH_SIZE, num_classes=NUM_CLASSES,
               image_dim=(IMAGE_SIZE, IMAGE_SIZE),
               is_train=True, is_shuffle=False, is_augment=False):

    """
    Custom data generator that gives [image array, feature vectors], label as output

    Args:
    image_filepaths: path to images (string)
    feature_vectors: LBP and HOG feature vector (numpy array)
    labels: labels or class (integer)
    is_train: True if input data is a train set
    is_shuffle: True to shuffle input data and if it is a train set
    is_augment: True to perform data augmentation and if it is a train set
    """

    # Define variables
    self.image_filepaths = image_filepaths
    self.feature_vectors = feature_vectors
    self.labels = labels

    self.image_dim = image_dim
    self.batch_size = batch_size
    self.num_classes = num_classes

    self.is_train = is_train
    self.is_augment = is_augment
    self.is_shuffle = is_shuffle

    self.indices = np.arange(len(image_filepaths))

    # Prevent shuffling and augmentation for generation of validation set
    if is_train == False:
      self.is_shuffle = False
      self.is_augment = False

    if self.is_train:
        self.on_epoch_end()

    # Define augmentations to perform on images
    self.iaa_augs = iaa.Sequential(
        [
            # Rotate the Images
            iaa.Affine(rotate=(-15, 15)),
            # Scale the Images
            iaa.Affine(scale=(0.5, 1.5)),
            # Flip the Images
            iaa.Fliplr(0.5),
            # Add Gaussian Blur
            iaa.GaussianBlur((1.0, 2.0)),
            # Add Gaussian Noise
            iaa.AdditiveGaussianNoise(scale=0.01*255, per_channel=True),
            # Multiply all pixels with a specific value
            iaa.Multiply((0.8, 1.5)),
        ]
    )

    print(f"Found {len(self.image_filepaths)} images belonging to {self.num_classes} classes")
    print(f"Found {len(self.feature_vectors)} feature vectors belonging to {self.num_classes} classes")

  def __len__(self):
    """
    Returns number of input data per batch
    """
    return int(np.floor(len(self.image_filepaths) / self.batch_size))

  def __getitem__(self, idx):
    """
    Generates a batch of data in the form of X, y
    """
    # Get indices of the batch
    indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]

    # Get list of IDs
    img_files_temp = [self.image_filepaths[i] for i in indices]

    # Generate data
    X, y = self.__data_generate(img_files_temp)

    return X, y

  def next(self):
    return self.__next__()

  def on_epoch_end(self):
    """
    Shuffle batch of data at every epoch
    """
    self.indices = np.arange(len(self.image_filepaths))
    if(self.is_train):
      np.random.shuffle(self.indices)
    else:
      pass

  def shuffle_data(self, x, y):
    """
    Shuffle data set

    x: a list of two numpy arrays [images, vectors],
    where images = x[0], vectors=x[1],
    images and vectors each is an array of size = self.batch_size
    """
    lam = np.random.beta(0.2, 0.4)
    ori_index = np.arange(int(len(x[0])))
    index_array = np.arange(int(len(x[0])))
    np.random.shuffle(index_array)

    shuffled_x = [lam * x[0][ori_index] + (1 - lam) * x[0][index_array],
                  lam * x[1][ori_index] + (1 - lam) * x[1][index_array]]
    shuffled_y = lam * y[ori_index] + (1 - lam) * y[index_array]

    return shuffled_x, shuffled_y

  def __data_generate(self, img_files_temp):
    """
    Generates batches of data
    """
    X_img = []
    X_features = []
    y = [0] * self.batch_size

    # Generate data
    for i, img_file in enumerate(img_files_temp):
      # Read image
      img = cv2.imread(img_file)

      # Apply image augmentation
      if self.is_augment:
          img = self.iaa_augs.augment_image(img)

      X_img.append(img)
      X_features.append(self.feature_vectors[img_file])

      y[i] = self.labels[img_file]

    X = [np.array(X_img, np.float32), np.array(X_features, np.float32)]
    y = np.asarray(y, dtype=np.float32)
    # y = tf.keras.utils.to_categorical(y, num_classes=self.num_classes)
    # print(y)

    if self.is_shuffle:
        X, y = self.shuffle_data(X, y)

    return X, y

In [ ]:
custom_train_datagen = CustomDataGenerator(image_filepaths=list(X_train_dict.keys()),
                                           feature_vectors=X_train_dict,
                                           labels=y_train_dict,
                                           is_train=True, is_augment=True, is_shuffle=True)
custom_val_datagen = CustomDataGenerator(image_filepaths=list(X_val_dict.keys()),
                                         feature_vectors=X_val_dict,
                                         labels=y_val_dict,
                                         is_train=False, is_augment=False, is_shuffle=False)

Found 400 images belonging to 40 classes
Found 400 feature vectors belonging to 40 classes
Found 100 images belonging to 40 classes
Found 100 feature vectors belonging to 40 classes


## Training Functions

In [ ]:
def train_fusion_model(
  model, model_name,
  train_gen, val_gen, batch_size,
  X_train,
  X_val,
  class_weights,
  result_dir,
  log_dir,
  version,
  workers, use_multiprocessing,
  initial_epoch=0, epochs=20,
  initial_value_threshold=None,
  additional_callbacks=[]):
  """
  Trains fusion model with inputs from custom data generator (CustomDataGenerator)
  """

  if not os.path.exists(os.path.join(result_dir, model_name)):
      os.makedirs(os.path.join(result_dir, model_name))

  if not os.path.exists(os.path.join(log_dir, model_name)):
      os.makedirs(os.path.join(log_dir, model_name))

  model_log_dir = os.path.join(log_dir, model_name, f'{model_name}_{version}')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(model_log_dir, histogram_freq=1)

  history = model.fit(
      train_gen,
      steps_per_epoch=len(train_gen.image_filepaths) // custom_train_datagen.batch_size,
      validation_data=val_gen,
      validation_steps=len(val_gen.image_filepaths) // val_gen.batch_size,
      class_weight=class_weights,
      epochs=epochs,
      initial_epoch=initial_epoch,
      workers=workers,
      use_multiprocessing=use_multiprocessing,
      verbose=1,
      callbacks=[
          tensorboard_callback,
          tf.keras.callbacks.TerminateOnNaN(),
          tf.keras.callbacks.CSVLogger(os.path.join(result_dir, model_name, f'{model_name}_{version}.log'), separator=',', append=True),
          tf.keras.callbacks.ModelCheckpoint(os.path.join(result_dir, model_name, f'{model_name}_{version}'),
                                monitor='val_loss', verbose=1,
                                save_best_only=True, mode='min', initial_value_threshold=initial_value_threshold)
      ] + additional_callbacks
  )

  return history, model

In [ ]:
def build_texture_mlp():
  """
  Builds and returns feed-forward model with pre-trained weights for Texture stream
  """

  # Define inputs
  input_texture = Input(shape=(27267,), name="texture_vector")

  y = Dense(64, activation='relu')(input_texture)
  y = Dense(28, activation='relu')(y)
  y = Dense(8, activation='relu')(y)
  y = Model(inputs=input_texture, outputs=y)

  return y

In [ ]:
def build_colour_cnn(base_model, model_name, is_trainable,
                     drop_out=0.25,
                     img_size=IMAGE_SIZE):

  """
  Builds and returns CNN model with pre-trained weights for Colour stream
  """

  print("Backbone model: ", model_name)

  # Define inputs
  input_image = Input(shape=(img_size, img_size, 3), name="coloured_image")

  # Freeze convolutional base
  if is_trainable == False:
    base_model.trainable = False

  # Instantiate pre-trained model
  if 'MobileNet' in model_name:
    # Pre-process input for model
    x = Lambda(tf.keras.applications.mobilenet_v2.preprocess_input)(input_image)

  # Instantiate pre-trained model
  elif 'ResNet50' in model_name:
    # Pre-process input for model
    x = Lambda(tf.keras.applications.resnet_v2.preprocess_input)(input_image)

  else:
    print('Backbone model not found')

  x = base_model(x, training=False)

  # Create fully connected layers
  x = GlobalAveragePooling2D()(x)
  # Add a Dropout layer
  x = Dropout(drop_out)(x)
  # Add Dense layers
  x = Dense(128, activation='relu')(x)
  # Add a Dropout layer
  x = Dropout(drop_out)(x)
  x = Dense(64, activation='relu')(x)

  # Create CNN model
  x = Model(inputs=input_image, outputs=x)

  return x

In [ ]:
def build_fusion_model(base_model, model_name, is_trainable,
                       cnn_drop_out=0.25,
                       num_classes=NUM_CLASSES,
                       img_size=IMAGE_SIZE):

  """
  Build fusion CNN-MLP model for mixed-data inputs and multi-output
  """

  # Instantiate model for Colour Stream
  x = build_colour_cnn(base_model, model_name, is_trainable,
                   drop_out=0.25,
                   img_size=IMAGE_SIZE)

  # Instantiate model for Texture Stream
  y = build_texture_mlp()

  # Concatenate two streams together
  combined = layers.concatenate([x.output, y.output])

  # Define joined Layer
  z = Dense(64, activation="relu")(combined)

  # Add Dense layer to output predictions
  z = Dense(num_classes, activation='softmax', name='predictions')(z)

  # Define the final model
  model = Model(inputs=[x.input, y.input], outputs=z)

  # Print model summary
  print(model.summary())

  return model

## MobileNet V2

In [ ]:
tf.keras.backend.clear_session()
K.clear_session()

In [ ]:
model_name = 'MobileNet_Multi'

# Load MobileNet V2 with ImageNet weights without last fully connected layers
mobilenet_base_model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                                      weights='imagenet',
                                      include_top=False)

mobilenet_multi_1 = build_fusion_model(mobilenet_base_model, model_name,
                             is_trainable=False)

Backbone model:  MobileNet_Multi
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 coloured_image (InputLayer)    [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda (Lambda)                (None, 224, 224, 3)  0           ['coloured_image[0][0]']         
                                                                                                  
 mobilenetv2_1.00_224 (Function  (None, 7, 7, 1280)  2257984     ['lambda[0][0]']                 
 al)                                                                                              
                                                           

In [ ]:
# Compile model
mobilenet_multi_1 = compile_model(mobilenet_multi_1, optimizer=Adam())

# Train model
mobilenet_multi_1_history, mobilenet_multi_1 = train_fusion_model(
    mobilenet_multi_1, model_name,
    train_gen=custom_train_datagen,
    val_gen=custom_val_datagen,
    X_train=list(X_train_dict.keys()),
    X_val=list(X_val_dict.keys()),
    class_weights=get_class_weights(list(y_train_dict.values()), is_encoded=True),
    result_dir=PP_MULTI_RESULT_DIR,
    log_dir=PP_MULTI_TB_DIR,
    additional_callbacks=add_callbacks(4),
    epochs=50,
    workers=2,
    use_multiprocessing=False,
    version='v1',
    batch_size=BATCH_SIZE)

Epoch 1/50
12/12 [==============================] - ETA: 0s - loss: 15.2667 - accuracy: 0.0156 - top_k_categorical_accuracy: 0.1198 - precision: 0.0252 - recall: 0.0054 - false_positives: 155.0000
Epoch 1: val_loss improved from inf to 4.01321, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/multi-results-pp/MobileNet_Multi/MobileNet_Multi_v1


12/12 [==============================] - 37s 3s/step - loss: 15.2667 - accuracy: 0.0156 - top_k_categorical_accuracy: 0.1198 - precision: 0.0252 - recall: 0.0054 - false_positives: 155.0000 - val_loss: 4.0132 - val_accuracy: 0.0312 - val_top_k_categorical_accuracy: 0.1458 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 2/50
12/12 [==============================] - ETA: 0s - loss: 3.8957 - accuracy: 0.0339 - top_k_categorical_accuracy: 0.1667 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 1.0000
Epoch 2: val_loss improved from 4.01321 to 3.65910, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/multi-results-pp/MobileNet_Multi/MobileNet_Multi_v1


12/12 [==============================] - 34s 3s/step - loss: 3.8957 - accuracy: 0.0339 - top_k_categorical_accuracy: 0.1667 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 1.0000 - val_loss: 3.6591 - val_accuracy: 0.0521 - val_top_k_categorical_accuracy: 0.1875 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 3/50
12/12 [==============================] - ETA: 0s - loss: 3.6678 - accuracy: 0.0312 - top_k_categorical_accuracy: 0.1979 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 3: val_loss improved from 3.65910 to 3.59491, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/multi-results-pp/MobileNet_Multi/MobileNet_Multi_v1


12/12 [==============================] - 35s 3s/step - loss: 3.6678 - accuracy: 0.0312 - top_k_categorical_accuracy: 0.1979 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.5949 - val_accuracy: 0.1042 - val_top_k_categorical_accuracy: 0.2708 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 4/50
12/12 [==============================] - ETA: 0s - loss: 3.6161 - accuracy: 0.0573 - top_k_categorical_accuracy: 0.2474 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 4: val_loss improved from 3.59491 to 3.55947, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/multi-results-pp/MobileNet_Multi/MobileNet_Multi_v1


12/12 [==============================] - 34s 3s/step - loss: 3.6161 - accuracy: 0.0573 - top_k_categorical_accuracy: 0.2474 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.5595 - val_accuracy: 0.0729 - val_top_k_categorical_accuracy: 0.3125 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 5/50
12/12 [==============================] - ETA: 0s - loss: 3.5670 - accuracy: 0.0729 - top_k_categorical_accuracy: 0.2526 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 5: val_loss improved from 3.55947 to 3.50029, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/multi-results-pp/MobileNet_Multi/MobileNet_Multi_v1


...

12/12 [==============================] - 29s 3s/step - loss: 2.2968 - accuracy: 0.4089 - top_k_categorical_accuracy: 0.7552 - precision: 0.7778 - recall: 0.0745 - false_positives: 16.0000 - val_loss: 2.2681 - val_accuracy: 0.3021 - val_top_k_categorical_accuracy: 0.6979 - val_precision: 0.8750 - val_recall: 0.1458 - val_false_positives: 2.0000 - lr: 0.0010
Epoch 31/50
12/12 [==============================] - ETA: 0s - loss: 1.9823 - accuracy: 0.4896 - top_k_categorical_accuracy: 0.8281 - precision: 0.8974 - recall: 0.0942 - false_positives: 8.0000
Epoch 31: val_loss improved from 2.26808 to 2.23451, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/multi-results-pp/MobileNet_Multi/MobileNet_Multi_v1


12/12 [==============================] - 32s 3s/step - loss: 1.9823 - accuracy: 0.4896 - top_k_categorical_accuracy: 0.8281 - precision: 0.8974 - recall: 0.0942 - false_positives: 8.0000 - val_loss: 2.2345 - val_accuracy: 0.3854 - val_top_k_categorical_accuracy: 0.7292 - val_precision: 0.6842 - val_recall: 0.1354 - val_false_positives: 6.0000 - lr: 0.0010
Epoch 32/50
12/12 [==============================] - ETA: 0s - loss: 1.8924 - accuracy: 0.4922 - top_k_categorical_accuracy: 0.8411 - precision: 0.8298 - recall: 0.1051 - false_positives: 16.0000
Epoch 32: val_loss did not improve from 2.23451
12/12 [==============================] - 10s 893ms/step - loss: 1.8924 - accuracy: 0.4922 - top_k_categorical_accuracy: 0.8411 - precision: 0.8298 - recall: 0.1051 - false_positives: 16.0000 - val_loss: 2.3622 - val_accuracy: 0.3542 - val_top_k_categorical_accuracy: 0.6458 - val_precision: 0.6667 - val_recall: 0.1667 - val_false_positives: 8.0000 - lr: 0.0010
Epoch 33/50
12/12 [=================

12/12 [==============================] - 32s 3s/step - loss: 1.9452 - accuracy: 0.4870 - top_k_categorical_accuracy: 0.8438 - precision: 0.8224 - recall: 0.1176 - false_positives: 19.0000 - val_loss: 2.1879 - val_accuracy: 0.3229 - val_top_k_categorical_accuracy: 0.7604 - val_precision: 0.6957 - val_recall: 0.1667 - val_false_positives: 7.0000 - lr: 0.0010
Epoch 35/50
12/12 [==============================] - ETA: 0s - loss: 1.8387 - accuracy: 0.4922 - top_k_categorical_accuracy: 0.8438 - precision: 0.8627 - recall: 0.1180 - false_positives: 14.0000
Epoch 35: val_loss improved from 2.18790 to 2.08261, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/multi-results-pp/MobileNet_Multi/MobileNet_Multi_v1


12/12 [==============================] - 32s 3s/step - loss: 1.8387 - accuracy: 0.4922 - top_k_categorical_accuracy: 0.8438 - precision: 0.8627 - recall: 0.1180 - false_positives: 14.0000 - val_loss: 2.0826 - val_accuracy: 0.3021 - val_top_k_categorical_accuracy: 0.7188 - val_precision: 0.7727 - val_recall: 0.1771 - val_false_positives: 5.0000 - lr: 0.0010
Epoch 36/50
12/12 [==============================] - ETA: 0s - loss: 1.7510 - accuracy: 0.4896 - top_k_categorical_accuracy: 0.8385 - precision: 0.8000 - recall: 0.1132 - false_positives: 21.0000
Epoch 36: val_loss did not improve from 2.08261
12/12 [==============================] - 10s 895ms/step - loss: 1.7510 - accuracy: 0.4896 - top_k_categorical_accuracy: 0.8385 - precision: 0.8000 - recall: 0.1132 - false_positives: 21.0000 - val_loss: 2.2262 - val_accuracy: 0.3125 - val_top_k_categorical_accuracy: 0.7292 - val_precision: 0.7500 - val_recall: 0.1875 - val_false_positives: 6.0000 - lr: 0.0010
Epoch 37/50
12/12 [================

12/12 [==============================] - 30s 3s/step - loss: 2.1568 - accuracy: 0.4193 - top_k_categorical_accuracy: 0.7917 - precision: 0.7407 - recall: 0.1071 - false_positives: 28.0000 - val_loss: 2.0645 - val_accuracy: 0.3125 - val_top_k_categorical_accuracy: 0.7500 - val_precision: 0.7200 - val_recall: 0.1875 - val_false_positives: 7.0000 - lr: 0.0010
Epoch 38/50
12/12 [==============================] - ETA: 0s - loss: 1.7630 - accuracy: 0.4896 - top_k_categorical_accuracy: 0.8672 - precision: 0.8661 - recall: 0.1309 - false_positives: 15.0000
Epoch 38: val_loss did not improve from 2.06452
12/12 [==============================] - 10s 884ms/step - loss: 1.7630 - accuracy: 0.4896 - top_k_categorical_accuracy: 0.8672 - precision: 0.8661 - recall: 0.1309 - false_positives: 15.0000 - val_loss: 2.1177 - val_accuracy: 0.3646 - val_top_k_categorical_accuracy: 0.7500 - val_precision: 0.6800 - val_recall: 0.1771 - val_false_positives: 8.0000 - lr: 0.0010
Epoch 39/50
12/12 [================

## ResNet50 V2

In [ ]:
model_name = 'ResNet50_Multi'

# Load ResNet50 V2 with ImageNet weights without last fully connected layers
resnet50_base_model = ResNet50V2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                                      weights='imagenet',
                                      include_top=False)

resnet50_multi_1 = build_fusion_model(resnet50_base_model, model_name,
                             is_trainable=False)

Backbone model:  ResNet50_Multi
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 coloured_image (InputLayer)    [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda_1 (Lambda)              (None, 224, 224, 3)  0           ['coloured_image[0][0]']         
                                                                                                  
 resnet50v2 (Functional)        (None, 7, 7, 2048)   23564800    ['lambda_1[0][0]']               
                                                                                                  
 global_average_pooling2d_1 (Gl  (None, 2048)        0      

In [ ]:
# Compile model
resnet50_multi_1 = compile_model(resnet50_multi_1, optimizer=Adam())

# Train model
resnet50_multi_1_history, resnet50_multi_1 = train_fusion_model(
    resnet50_multi_1, model_name,
    train_gen=custom_train_datagen,
    val_gen=custom_val_datagen,
    X_train=list(X_train_dict.keys()),
    X_val=list(X_val_dict.keys()),
    class_weights=get_class_weights(list(y_train_dict.values()), is_encoded=True),
    result_dir=PP_MULTI_RESULT_DIR,
    log_dir=PP_MULTI_TB_DIR,
    additional_callbacks=add_callbacks(4),
    epochs=50,
    workers=2,
    use_multiprocessing=False,
    version='v1',
    batch_size=BATCH_SIZE)

Epoch 1/50
12/12 [==============================] - ETA: 0s - loss: 12.2700 - accuracy: 0.0391 - top_k_categorical_accuracy: 0.1198 - precision: 0.0559 - recall: 0.0107 - false_positives: 135.0000
Epoch 1: val_loss improved from inf to 3.92586, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/multi-results-pp/ResNet50_Multi/ResNet50_Multi_v1


12/12 [==============================] - 39s 3s/step - loss: 12.2700 - accuracy: 0.0391 - top_k_categorical_accuracy: 0.1198 - precision: 0.0559 - recall: 0.0107 - false_positives: 135.0000 - val_loss: 3.9259 - val_accuracy: 0.0417 - val_top_k_categorical_accuracy: 0.1562 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 2/50
12/12 [==============================] - ETA: 0s - loss: 3.7927 - accuracy: 0.0339 - top_k_categorical_accuracy: 0.1615 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 2: val_loss improved from 3.92586 to 3.59278, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/multi-results-pp/ResNet50_Multi/ResNet50_Multi_v1


12/12 [==============================] - 34s 3s/step - loss: 3.7927 - accuracy: 0.0339 - top_k_categorical_accuracy: 0.1615 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.5928 - val_accuracy: 0.0833 - val_top_k_categorical_accuracy: 0.2500 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 3/50
12/12 [==============================] - ETA: 0s - loss: 3.6687 - accuracy: 0.0312 - top_k_categorical_accuracy: 0.1849 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 3: val_loss improved from 3.59278 to 3.57045, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/multi-results-pp/ResNet50_Multi/ResNet50_Multi_v1


12/12 [==============================] - 34s 3s/step - loss: 3.6687 - accuracy: 0.0312 - top_k_categorical_accuracy: 0.1849 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.5705 - val_accuracy: 0.0521 - val_top_k_categorical_accuracy: 0.2812 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 4/50
12/12 [==============================] - ETA: 0s - loss: 3.5855 - accuracy: 0.0755 - top_k_categorical_accuracy: 0.2188 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 4: val_loss improved from 3.57045 to 3.50434, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/multi-results-pp/ResNet50_Multi/ResNet50_Multi_v1


12/12 [==============================] - 34s 3s/step - loss: 3.5855 - accuracy: 0.0755 - top_k_categorical_accuracy: 0.2188 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 3.5043 - val_accuracy: 0.1042 - val_top_k_categorical_accuracy: 0.3229 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 5/50
12/12 [==============================] - ETA: 0s - loss: 3.4812 - accuracy: 0.0938 - top_k_categorical_accuracy: 0.3021 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 5: val_loss improved from 3.50434 to 3.41385, saving model to /content/drive/MyDrive/uploaded_datasets/pill-recognition/all-results/multi-results-pp/ResNet50_Multi/ResNet50_Multi_v1


...

12/12 [==============================] - 35s 3s/step - loss: 2.0564 - accuracy: 0.4453 - top_k_categorical_accuracy: 0.8307 - precision: 0.8252 - recall: 0.1127 - false_positives: 18.0000 - val_loss: 1.7652 - val_accuracy: 0.4583 - val_top_k_categorical_accuracy: 0.8646 - val_precision: 0.7619 - val_recall: 0.1667 - val_false_positives: 5.0000 - lr: 0.0010
Epoch 24/50
12/12 [==============================] - ETA: 0s - loss: 1.7891 - accuracy: 0.4714 - top_k_categorical_accuracy: 0.8802 - precision: 0.7917 - recall: 0.1017 - false_positives: 20.0000
Epoch 24: val_loss did not improve from 1.76519
12/12 [==============================] - 11s 915ms/step - loss: 1.7891 - accuracy: 0.4714 - top_k_categorical_accuracy: 0.8802 - precision: 0.7917 - recall: 0.1017 - false_positives: 20.0000 - val_loss: 1.7766 - val_accuracy: 0.4271 - val_top_k_categorical_accuracy: 0.8229 - val_precision: 0.7143 - val_recall: 0.2083 - val_false_positives: 8.0000 - lr: 0.0010
Epoch 25/50
12/12 [================

12/12 [==============================] - 35s 3s/step - loss: 1.9937 - accuracy: 0.4453 - top_k_categorical_accuracy: 0.8750 - precision: 0.8230 - recall: 0.1250 - false_positives: 20.0000 - val_loss: 1.7049 - val_accuracy: 0.4479 - val_top_k_categorical_accuracy: 0.8333 - val_precision: 0.8750 - val_recall: 0.2188 - val_false_positives: 3.0000 - lr: 0.0010
Epoch 26/50
12/12 [==============================] - ETA: 0s - loss: 1.8648 - accuracy: 0.5234 - top_k_categorical_accuracy: 0.8594 - precision: 0.8718 - recall: 0.1360 - false_positives: 15.0000
Epoch 26: val_loss did not improve from 1.70488
12/12 [==============================] - 10s 851ms/step - loss: 1.8648 - accuracy: 0.5234 - top_k_categorical_accuracy: 0.8594 - precision: 0.8718 - recall: 0.1360 - false_positives: 15.0000 - val_loss: 1.7566 - val_accuracy: 0.5000 - val_top_k_categorical_accuracy: 0.8333 - val_precision: 0.8846 - val_recall: 0.2396 - val_false_positives: 3.0000 - lr: 0.0010
Epoch 27/50
12/12 [================

12/12 [==============================] - 34s 3s/step - loss: 2.2695 - accuracy: 0.4375 - top_k_categorical_accuracy: 0.7839 - precision: 0.8242 - recall: 0.1011 - false_positives: 16.0000 - val_loss: 1.7042 - val_accuracy: 0.4792 - val_top_k_categorical_accuracy: 0.8125 - val_precision: 0.8571 - val_recall: 0.1875 - val_false_positives: 3.0000 - lr: 0.0010
Epoch 28/50
12/12 [==============================] - ETA: 0s - loss: 1.7005 - accuracy: 0.5260 - top_k_categorical_accuracy: 0.8750 - precision: 0.8318 - recall: 0.1195 - false_positives: 18.0000
Epoch 28: val_loss did not improve from 1.70420
12/12 [==============================] - 10s 847ms/step - loss: 1.7005 - accuracy: 0.5260 - top_k_categorical_accuracy: 0.8750 - precision: 0.8318 - recall: 0.1195 - false_positives: 18.0000 - val_loss: 1.7232 - val_accuracy: 0.4271 - val_top_k_categorical_accuracy: 0.8542 - val_precision: 0.6452 - val_recall: 0.2083 - val_false_positives: 11.0000 - lr: 0.0010
Epoch 29/50
12/12 [===============

12/12 [==============================] - 34s 3s/step - loss: 2.0897 - accuracy: 0.4714 - top_k_categorical_accuracy: 0.8464 - precision: 0.8083 - recall: 0.1295 - false_positives: 23.0000 - val_loss: 1.6023 - val_accuracy: 0.4688 - val_top_k_categorical_accuracy: 0.8438 - val_precision: 0.8621 - val_recall: 0.2604 - val_false_positives: 4.0000 - lr: 0.0010
Epoch 30/50
12/12 [==============================] - ETA: 0s - loss: 1.8448 - accuracy: 0.5234 - top_k_categorical_accuracy: 0.8802 - precision: 0.8899 - recall: 0.1292 - false_positives: 12.0000
Epoch 30: val_loss did not improve from 1.60232
12/12 [==============================] - 11s 912ms/step - loss: 1.8448 - accuracy: 0.5234 - top_k_categorical_accuracy: 0.8802 - precision: 0.8899 - recall: 0.1292 - false_positives: 12.0000 - val_loss: 1.6665 - val_accuracy: 0.4583 - val_top_k_categorical_accuracy: 0.8750 - val_precision: 0.8148 - val_recall: 0.2292 - val_false_positives: 5.0000 - lr: 0.0010
Epoch 31/50
12/12 [================

12/12 [==============================] - 35s 3s/step - loss: 2.0719 - accuracy: 0.4583 - top_k_categorical_accuracy: 0.7865 - precision: 0.8962 - recall: 0.1272 - false_positives: 11.0000 - val_loss: 1.5663 - val_accuracy: 0.5104 - val_top_k_categorical_accuracy: 0.8854 - val_precision: 0.7586 - val_recall: 0.2292 - val_false_positives: 7.0000 - lr: 0.0010
Epoch 32/50
12/12 [==============================] - ETA: 0s - loss: 1.8736 - accuracy: 0.5000 - top_k_categorical_accuracy: 0.8620 - precision: 0.8889 - recall: 0.1283 - false_positives: 12.0000
Epoch 32: val_loss did not improve from 1.56626
12/12 [==============================] - 10s 856ms/step - loss: 1.8736 - accuracy: 0.5000 - top_k_categorical_accuracy: 0.8620 - precision: 0.8889 - recall: 0.1283 - false_positives: 12.0000 - val_loss: 1.6277 - val_accuracy: 0.4688 - val_top_k_categorical_accuracy: 0.8646 - val_precision: 0.7647 - val_recall: 0.2708 - val_false_positives: 8.0000 - lr: 0.0010
Epoch 33/50
12/12 [================

## Learning Curves

In [ ]:
# %tensorboard --logdir /content/drive/MyDrive/uploaded_datasets/pill-classification/all-results/multi-tb-pp/MobileNet_Multi/MobileNet_Multi_bs_16

# %tensorboard --logdir /content/drive/MyDrive/uploaded_datasets/pill-classification/all-results/multi-tb-pp/ResNet50_Multi/ResNet50_Multi_bs_16

# Evaluate on Test Set

## Data Generator for Test Set

In [ ]:
custom_test_datagen = CustomDataGenerator(image_filepaths=list(X_test_dict.keys()),
                                         feature_vectors=X_test_dict,
                                         labels=y_test_dict,
                                         is_train=False, is_augment=False, is_shuffle=False)

Found 56 images belonging to 40 classes
Found 56 feature vectors belonging to 40 classes


In [ ]:
evaluation_metrics = mobilenet_multi_1.metrics_names
print(evaluation_metrics)

['loss', 'accuracy', 'top_k_categorical_accuracy', 'precision', 'recall', 'false_positives']


In [ ]:
mobilenet_eval_results = mobilenet_multi_1.evaluate(custom_test_datagen, return_dict=True, verbose=1)
resnet50_eval_results = resnet50_multi_1.evaluate(custom_test_datagen, return_dict=True, verbose=1)

1/1 [==============================] - 0s 257ms/step - loss: 1.7049 - accuracy: 0.4688 - top_k_categorical_accuracy: 0.8125 - precision: 0.8571 - recall: 0.1875 - false_positives: 1.0000


In [ ]:
def compare_results(model_names, result_array):
  print("Model Performance\n")

  for model_name, results in zip(model_names, result_array):
    print("======================== {} ========================".format(model_name))
    print(pd.DataFrame(results.items(), columns=['metric', 'score']), "\n\n")

In [ ]:
compare_results(['MobileNet', 'ResNet50'], [mobilenet_eval_results, resnet50_eval_results])

Model Performance

======================== MobileNet ========================
                       metric     score
0                        loss  2.226971
1                    accuracy  0.281250
2  top_k_categorical_accuracy  0.750000
3                   precision  0.625000
4                      recall  0.156250
5             false_positives  3.000000 


======================== ResNet50 ========================
                       metric     score
0                        loss  1.704935
1                    accuracy  0.468750
2  top_k_categorical_accuracy  0.812500
3                   precision  0.857143
4                      recall  0.187500
5             false_positives  1.000000 


